In [21]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langchain.chat_models import init_chat_model
from typing import Literal


In [22]:
class AgentsState(MessagesState):
    current_agent: str
    transfered_by: str


llm = init_chat_model(model="openai:gpt-5-nano-2025-08-07")

In [23]:
def make_agent(prompt, agent_name, tools):
    def agent_node(state: AgentsState):
        llm_with_tools = llm.bind_tools(tools)
        messages = [SystemMessage(content=prompt)] + state["messages"]
        response = llm_with_tools.invoke(messages)
        # response = llm_with_tools.invoke(
        #     f"""
        # {prompt}

        # Conversation History:
        # {state["messages"]}
        # """
        # )
        return {"messages": [response]}

    agent_builder = StateGraph(AgentsState)

    agent_builder.add_node("agent", agent_node)
    agent_builder.add_node("tools", ToolNode(tools=tools))

    agent_builder.add_edge(START, "agent")
    agent_builder.add_conditional_edges("agent", tools_condition)
    agent_builder.add_edge("tools", "agent")

    return agent_builder.compile()

In [24]:
@tool
def handoff_tool(
    transfer_to: Literal["korean_agent", "greek_agent", "english_agent"],
    transfered_by: str,
):
    """
    Handoff to another agent.
    Use this tool when the customer speaks a language that you don't understand.
    """
    return Command(
        update={
            "current_agent": transfer_to,
            "transfered_by": transfered_by,
        },
        goto=transfer_to,
        graph=Command.PARENT,  # 현재 그래프가 아닌, 한단계 위의 그래프로 통제권을 넘긴다
    )

In [25]:
graph_builder = StateGraph(AgentsState)

# 각 에이전트 노드 생성
agents = [
    {
        "name": "korean_agent",
        "prompt": "You are a Korean customer support agent. You only speak and understand Korean. If user speaks other languages, handoff to relevant agent.",
    },
    {
        "name": "greek_agent",
        "prompt": "You are a Greek customer support agent. You only speak and understand Greek. If user speaks other languages, handoff to relevant agent.",
    },
    {
        "name": "english_agent",
        "prompt": "You are an English customer support agent. You only speak and understand English. If user speaks other languages, handoff to relevant agent.",
    },
]

for agent_info in agents:
    graph_builder.add_node(
        agent_info["name"],
        make_agent(
            prompt=agent_info["prompt"],
            agent_name=agent_info["name"],
            tools=[handoff_tool],
        ),
    )

# 시작점 설정
graph_builder.add_edge(START, "korean_agent")

graph = graph_builder.compile()

In [26]:
for event in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "안녕하세요. 로그인이 되지 않고있어",
            },
        ]
    },
    stream_mode="updates",
):
    print(event)


{'korean_agent': {'messages': [HumanMessage(content='안녕하세요. 로그인이 되지 않고있어', additional_kwargs={}, response_metadata={}, id='581c45ab-fbd2-4e4b-89a7-5c2b1c5efdd4'), AIMessage(content='도와드리겠습니다. 먼저 몇 가지 정보를 확인드릴게요.\n\n상황 파악용 점검 가이드\n- 어떤 서비스(웹/앱)에서 로그인하려고 하나요? 예: 사이트 이름이나 앱 이름\n- 현재 어떤 화면이나 에러 메시지가 나오나요? 예: "비밀번호가 올바르지 않습니다" or "계정이 잠겼습니다" 등\n- 사용 중인 기기와 브라우저/앱 버전은 무엇인가요? (예: Windows/Chrome, iPhone iOS 17 등)\n- 최근에 비밀번호를 바꿨나요? 혹은 계정 정보(이메일/아이디) 변경 이력이 있나요?\n- 2단계 인증(Two-factor) 설정이 되어 있나요? 인증 코드가 필요한 상태인가요?\n\n권장하는 기본 조치\n- 먼저 입력하신 아이디/이메일과 비밀번호를 정확히 확인하고, Caps Lock이나 키보드 언어 설정이 의도대로 되어 있는지 점검하기\n- 비밀번호 재설정 시도\n  - 로그인 화면에서 “비밀번호를 잊으셨나요?” 또는 “비밀번호 재설정” 링크를 클릭\n  - 등록된 이메일/전화번호로 재설정 코드나 링크를 받으면, 안내에 따라 새 비밀번호 설정\n  - 새 비밀번호는 8자 이상으로 대소문자, 숫자, 특수문자 조합을 권장\n  - 이메일 수신함의 스팸/프로모션 탭도 확인\n- 캐시/쿠키 문제일 수 있으니, 다른 브라우저나 사파리/엣지 등의 시도, 또는 브라우저 캐시·쿠키 삭제 후 재로그인 시도\n- 2단계 인증 문제가 있다면, 인증 앱의 코드 입력 또는 SMS 코드를 사용해 다시 로그인 시도\n\n도와드리기 위한 추가 정보가 필요합니다\n- 로그인하려는 서비스 이름과 화면의 구체적인 에러 메시지\n- 사용 중인 기기와 브라우저/앱 버전\n- 